In [4]:
# Import Dependencies
import requests
import json
from bs4 import BeautifulSoup
import time
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from pandas import to_datetime
import numpy as np

In [6]:
# Read CSV File
author_df = pd.read_csv("cleaned_tables/final_book.csv")
author_df

,isbn,title,language,main author,description,pages,publisher,publishedDate,yearPublished,rating,numRatings,likedPercent,price
0,9780062348678,Simon vs. the Homo Sapiens Agenda,English,Becky Albertalli,Sixteen-year-old and not-so-openly gay Simon S...,303,Balzer + Bray,2015-04-07,2015,4.29,288285,96.0,11.51
1,B0745KTJSY,Christy,English,Catherine Marshall,The train taking nineteen-year-old teacher Chr...,512,Evergreen Farm,2017-10-03,2017,4.23,49453,96.0,0.00
2,9781408857908,A Court of Wings and Ruin,English,Sarah J. Maas,Looming war threatens all Feyre holds dear in ...,699,Bloomsbury Childrens Books,2017-05-02,2017,4.44,253163,96.0,10.58
3,9781786495259,Call Me By Your Name,English,André Aciman,Call Me by Your Name is the story of a sudden ...,248,Atlantic Books,2017-09-21,2017,4.23,210409,94.0,4.20
4,9780143129554,White Noise,English,Don DeLillo,"Part of the Penguin Orange Collection, a limit...",320,Penguin Classics,2016-09-06,2016,3.87,86742,90.0,8.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3814,9781250078285,"Wires and Nerve, Volume 2: Gone Rogue",English,Marissa Meyer,Iko – an audacious android and best friend to ...,324,Feiwel Friends,2018-01-30,2018,4.23,11512,97.0,11.66
3815,9781447256274,The Burning Page,English,Genevieve Cogman,The third title in Genevieve Cogman's clever a...,358,Pan,2016-12-15,2016,4.05,12808,98.0,3.58
3816,B01KZOA32U,Dead Stop,English,Barbara Nickless,The search for a missing child leads a courage...,398,Thomas & Mercer,2017-10-03,2017,4.51,7088,99.0,0.00
3817,B00XKC1KTK,See You Never,English,Misty Jae Ogert,"Harper Quinn has it all, a handsome husband, B...",421,Moon and Back Books,2015-05-11,2015,3.73,74,82.0,0.00


In [8]:
unique_author_list = author_df['main author'].unique().tolist()
unique_author_list

['Becky Albertalli ',
 'Catherine Marshall',
 'Sarah J. Maas ',
 'André Aciman ',
 'Don DeLillo',
 'Aimee Cabo Nikolov ',
 'Hanya Yanagihara ',
 'V.E. Schwab',
 'Nicola Yoon ',
 'Marissa Meyer ',
 'Mia Sheridan ',
 'Cassandra Clare ',
 'Kathleen Zamboni McCormick ',
 'Kenneth Jarrett Singleton',
 'Fernando Pessoa',
 'Maria Nhambu ',
 'Anne Willow ',
 'Amie Kaufman ',
 'Colleen Hoover ',
 'Laura Joffe Numeroff ',
 'Ransom Riggs',
 'Delia Owens ',
 'M. Robinson',
 'Afroz Alam ',
 'Lawrence Hill',
 'John Green ',
 'John Tiffany ',
 'Kiera Cass ',
 'Tara Westover ',
 'Michel Guyon ',
 'Liane Moriarty ',
 'Victoria Aveyard ',
 'Ammar Habib ',
 'Rohith S. Katbamna ',
 'Stephanie Garber ',
 'Madeline Miller ',
 'Jason Latshaw ',
 'Kirsten Fullmer ',
 'Gail Honeyman',
 'Garth Nix ',
 'Paul Kalanithi',
 'Margaret Wise Brown',
 'Gillian Dance ',
 'Naomi Novik ',
 'Samuel R. George',
 'Rick Riordan ',
 'J.K. Franko ',
 'Taylor Jenkins Reid ',
 'Ted Bernal Guevara ',
 'Blake Crouch ',
 'Renée Ahdi

In [10]:
# Web scraping wikepedia page
# Base url
base_url = "https://en.wikipedia.org/wiki/"
author_dob = {}
for author in unique_author_list:
    # create url along with author name
    url = base_url + author
    # Retrieve page with the requests module
    response = requests.get(url)
    try:
        # Create BeautifulSoup object; parse with 'html.parser'
        soup = BeautifulSoup(response.text, 'html.parser')
        author_dob[author]= soup.find('span', class_='bday').text
    except AttributeError:
        author_dob[author]= "none"

author_dob

{'Becky Albertalli ': '1982-11-17',
 'Catherine Marshall': '1914-09-27',
 'Sarah J. Maas ': '1986-03-05',
 'André Aciman ': '1951-01-02',
 'Don DeLillo': '1936-11-20',
 'Aimee Cabo Nikolov ': 'none',
 'Hanya Yanagihara ': 'none',
 'V.E. Schwab': '1987-07-07',
 'Nicola Yoon ': 'none',
 'Marissa Meyer ': '1984-02-19',
 'Mia Sheridan ': 'none',
 'Cassandra Clare ': '1973-07-27',
 'Kathleen Zamboni McCormick ': 'none',
 'Kenneth Jarrett Singleton': 'none',
 'Fernando Pessoa': '1888-06-13',
 'Maria Nhambu ': 'none',
 'Anne Willow ': 'none',
 'Amie Kaufman ': 'none',
 'Colleen Hoover ': '1979-12-11',
 'Laura Joffe Numeroff ': '1953-07-14',
 'Ransom Riggs': '1979-02-03',
 'Delia Owens ': 'none',
 'M. Robinson': 'none',
 'Afroz Alam ': 'none',
 'Lawrence Hill': '1957-01-24',
 'John Green ': '1977-08-24',
 'John Tiffany ': 'none',
 'Kiera Cass ': '1981-05-19',
 'Tara Westover ': '1986-09-30',
 'Michel Guyon ': 'none',
 'Liane Moriarty ': '1966-11-15',
 'Victoria Aveyard ': '1990-07-27',
 'Ammar

In [11]:
# Converting dictionary to dataframe
dbf_dob = pd.DataFrame.from_dict(author_dob, orient ='index')

In [12]:
# Saving to csv file
dbf_dob.to_csv('scraped_author_data/birth_date1.csv', index=True)

In [18]:
# author dataframe
author_info_df = pd.read_csv('scraped_author_data/birth_date1.csv')
author_info_df = author_info_df.rename(columns={"Unnamed: 0":"main author","0":"dob"})
author_info_df

,main author,dob
0,Becky Albertalli,1982-11-17
1,Catherine Marshall,1914-09-27
2,Sarah J. Maas,1986-03-05
3,André Aciman,1951-01-02
4,Don DeLillo,1936-11-20
...,...,...
2701,Jeanne St. James,none
2702,Darynda Jones,none
2703,Barbara Nickless,none
2704,Misty Jae Ogert,none
